## Construcción de bases de datos

In [53]:
import requests
import pandas as pd
import locale
import datetime
import os
import py7zr
from io import StringIO, BytesIO
from functools import reduce
import eurostat


locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  # Unix/Linux

'es_ES.UTF-8'

In [54]:
# Obtener series de PMI (Desde MQL5)
def obtener_series_PMI():

    url_services = "https://www.mql5.com/en/economic-calendar/spain/markit-services-pmi/export"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Descargar el archivo
    response = requests.get(url_services, headers=headers)
    response.raise_for_status()

    # Leer el CSV con separador TAB
    csv_text = response.content.decode("utf-8")
    df = pd.read_csv(StringIO(csv_text), sep='\t')

    # Renombrar columnas y convertir tipos
    df = df.rename(columns={
        "Date": "Fecha",
        "ActualValue": "PMI_servicios"
    })

    # Conversión de variables
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y.%m.%d") + pd.offsets.MonthBegin(0)
    df["PMI_servicios"] = pd.to_numeric(df["PMI_servicios"], errors='coerce')

    # Guardar fechas

    services_pmi = df[["Fecha", "PMI_servicios"]].sort_values("Fecha")

    services_pmi = services_pmi.reset_index(drop=True)

    url_manufacturing = "https://www.mql5.com/en/economic-calendar/spain/markit-services-pmi/export"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Descargar el archivo
    response = requests.get(url_manufacturing, headers=headers)
    response.raise_for_status()

    # Leer el CSV con separador TAB
    csv_text = response.content.decode("utf-8")
    df = pd.read_csv(StringIO(csv_text), sep='\t')

    # Renombrar columnas y convertir tipos
    df = df.rename(columns={
        "Date": "Fecha",
        "ActualValue": "PMI_manufactura"
    })

    # Conversión de variables
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y.%m.%d") + pd.offsets.MonthBegin(0)
    df["PMI_manufactura"] = pd.to_numeric(df["PMI_manufactura"], errors='coerce')

    # Guardar fechas

    manufacturing_pmi = df[["Fecha", "PMI_manufactura"]].sort_values("Fecha")

    manufacturing_pmi = manufacturing_pmi.reset_index(drop=True)

    return services_pmi, manufacturing_pmi

In [55]:
# Obtener series de Consumo Aparente de Cemento (del Ministerio de Industria y Turismo)

def obtener_series_cemento():

    # Se debe hacer una unión entre la serie histórica de 1992 hasta diciembre de 2023, y una serie ampliada con cambios metodológicos

    url_1 = "https://industria.gob.es/es-es/estadisticas/Documents/Estadistica-Cemento/serie-historica-desde-1992/Estadistica-cemento-serie-1992-2023.xlsx"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Descarga de la serie histórica de 1992 a 2023
    response_1 = requests.get(url_1, headers=headers, verify=False)
    response_1.raise_for_status()

    serie_1 = pd.read_excel(BytesIO(response_1.content), skiprows=11)

    df_1 = serie_1[["FECHA", "CONSUMO APARENTE DE CEMENTO"]]

    # Filtraje de las fechas compartidas con la segunda serie:

    df_1 = df_1[df_1["FECHA"] < "2023-04-01"]

    url_2 = "https://industria.gob.es/es-es/estadisticas/Documents/Estadistica-Cemento/principales-resultados/estadisticas-cemento-2025-base-no-ampliada.xlsx"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Descarga de la serie histórica de 1992 a 2023
    response_2 = requests.get(url_2, headers=headers, verify=False)
    response_2.raise_for_status()

    serie_2 = pd.read_excel(BytesIO(response_2.content), skiprows=6)

    df_2 = serie_2[["PERIODO", "Consumo Aparente de Cemento"]]

    df_2 = df_2.rename(columns={'PERIODO': 'FECHA', 'Consumo Aparente de Cemento': 'CONSUMO APARENTE DE CEMENTO'})

    series_cemento = pd.concat([df_1, df_2], ignore_index=True)

    series_cemento = series_cemento.rename(columns={'FECHA': 'Fecha', 'CONSUMO APARENTE DE CEMENTO': 'Cemento'})

    return series_cemento

In [56]:
# Obtención de Trabajadores en alta laboral en la Seguridad Social. Media Mensual. CVEC 

def obtener_afiliados_seguridad_social():
    """Obtiene el número de afiliados en alta laboral en la Seguridad Social (mensual)."""
    # Supongamos que existe un CSV publicado en una URL fija con los datos históricos
    url = "https://www.seg-social.es/wps/wcm/connect/wss/5ccf558a-868f-48b3-b832-04fe9f524960/19_Serie+afiliaci%C3%B3n+media+por+reg%C3%ADmenes+2001-202503.xlsx?MOD=AJPERES&CONVERT_TO=linktext&CACHEID=ROOTWORKSPACE.Z18_2G50H38209D640QTQ57OVB2000-5ccf558a-868f-48b3-b832-04fe9f524960-pnRfMll"
    
    df = pd.read_excel(url, skiprows=1, usecols=['Régimen', 'TOTAL SISTEMA'])
    
    # Eliminar segunda línea vacía
    df = df.drop(axis=0, index=0)

    # Eliminar líneas sin valores (son notas)
    df = df.dropna(axis=0, how="any")

    # Pasar a formato fecha y eliminar columna
    df['Fecha'] = pd.to_datetime(df['Régimen'], format='%B %Y')
    df = df.drop(axis=1, columns="Régimen")
    
    # Cambio de orden
    series_afiliados = df[['Fecha', 'TOTAL SISTEMA']]

    series_afiliados = series_afiliados.rename(columns={'TOTAL SISTEMA': 'Afiliados_total'})

    return series_afiliados

In [57]:
# Obtener series del INE: PIB, IASS, EPA

# Inspiración: https://danielsancheznet.wordpress.com/2020/10/20/extraccion_datos_ine_api/ 

def obtener_ine_series():
    
    def ine_request(ine_code):
        resultados = 999
        path_template = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{cod_serie}?nult={n_ult_datos}'
        path = path_template.format(cod_serie=ine_code, n_ult_datos=resultados)
        json_request = requests.get(path).json()        
        return json_request

    df_codigos = pd.read_excel('ine_codes.xlsx')

    for idx, fila in df_codigos.iterrows():
        codigo = fila['ine_code']
        nombre_variable_archivo = fila['Variable']

        try:
            datos = ine_request(codigo)
            
            fecha_lista = []
            valor_lista = []

            for dato in datos['Data']:
                fecha = datetime.date.fromtimestamp(dato['Fecha'] // 1000)
                fecha_lista.append(fecha)
                valor_lista.append(dato['Valor'])

            df_variable = pd.DataFrame({
                'Fecha': fecha_lista,
                'Dato': valor_lista
            })

            df_variable = df_variable.rename(columns={'Dato': nombre_variable_archivo})

            nombre_archivo = f"{nombre_variable_archivo.strip().replace(' ', '_')}.xlsx"
            df_variable.to_excel(nombre_archivo, index=False)
            print(f"Guardado: {nombre_archivo}")

        except Exception as e:
            print(f"Error al procesar código {codigo} ({nombre_variable_archivo}): {e}")



In [58]:
series_ine = obtener_ine_series() # Obtención de variables del INE

serie_IASS = pd.read_excel("IASS.xlsx")

serie_IPI = pd.read_excel("IPI.xlsx")

serie_PIB = pd.read_excel("PIB.xlsx")

serie_ocupados = pd.read_excel("EPA.xlsx")

Guardado: IPI.xlsx
Guardado: EPA.xlsx
Guardado: PIB.xlsx
Guardado: IASS.xlsx
Guardado: IPI_consumo.xlsx
Guardado: IPI_equipo.xlsx
Guardado: Retail.xlsx
Guardado: Entradas_turistas.xlsx
Guardado: Construccion_indice.xlsx
Guardado: Compraventa.xlsx
Guardado: PIB.xlsx


In [59]:
# Obtener series de la AEAT: Rendimientos, Vental totales, Importaciones

def obtener_series_AEAT():

    zip_file_url = "https://sede.agenciatributaria.gob.es/static_files/AEAT/Estudios/Estadisticas/Informes_Estadisticos/Informe_VESGE/BdDVesge.7z"
    filename = os.path.basename(zip_file_url)

    def descargar_archivo():
        response = requests.get(zip_file_url, stream=True)
        if response.status_code == 200:
            with open(filename, 'wb') as out:
                for chunk in response.iter_content(chunk_size=8192):
                    out.write(chunk)
            print("Descarga completada.")
        else:
            raise Exception(f"Error en la descarga: {response.status_code}")

    # Si el archivo ya existe
    if os.path.exists(filename):
        respuesta = input(f"El archivo '{filename}' ya existe. ¿Deseas sobrescribirlo? [s/n]: ").strip().lower()
        if respuesta == 's':
            print("Sobrescribiendo archivo existente...")
            descargar_archivo()
        else:
            print("Descarga cancelada. Se mantiene el archivo existente.")
    else:
        descargar_archivo()

    archivo_7z = "BdDVesge.7z"
    extract_dir = "BdDVesge_extract"
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer con py7zr
    with py7zr.SevenZipFile(archivo_7z, mode='r') as archive:
        archive.extractall(path=extract_dir)

    # Mostrar todos los archivos extraídos
    for root, dirs, files in os.walk(extract_dir):
        for name in files:
            print("Archivo extraído:", os.path.join(root, name))

    # Buscar archivo .txt extraído (en caso de que el nombre varíe)
    txt_files = [f for f in os.listdir(extract_dir) if f.endswith('.txt')]
    print("Archivos .txt encontrados:", txt_files)

    # Ruta al fichero esperado
    ruta_txt = os.path.join(extract_dir, txt_files[0])  # si hay solo uno

    # Leer el contenido
    df = pd.read_csv(ruta_txt, sep=';', encoding='latin1')

    # Extracción de series 

    # Variable 2: Ventas totales de Grandes Empresas
    serie_ventas = df[(df["VARIABLE"] == 2) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 10: Rendimientos
    serie_rendimientos = df[(df["VARIABLE"] == 10) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 11: Rendimiento Bruto Medio
    serie_rendimientos = df[(df["VARIABLE"] == 11) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 4: Importaciones totales
    serie_importaciones = df[(df["VARIABLE"] == 4) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 3: Exportaciones totales
    serie_exportaciones = df[(df["VARIABLE"] == 3) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 2: Ventas totales + 14: Construcción: Ventas totale de la construcción
    serie_ventas_construcción = df[(df["VARIABLE"] == 2) & (df["LITERAL"] == 14) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 9: Perceptores
    serie_perceptores = df[(df["VARIABLE"] == 9) & (df["LITERAL"] == 29) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Variable 12_ Ventas interiores por destino
    serie_consumo = df[(df["VARIABLE"] == 12) & (df["LITERAL"] == 1) & (df["FREC"] == 1) & (df["DATO1"] == 1) & (df["DATO2"] == 2)]

    # Extracción de períodos

    serie_ventas = serie_ventas.reset_index(drop=True)
    serie_rendimientos = serie_rendimientos.reset_index(drop=True)
    serie_importaciones = serie_importaciones.reset_index(drop=True)
    serie_exportaciones = serie_exportaciones.reset_index(drop=True)
    serie_ventas_construcción = serie_ventas_construcción.reset_index(drop=True)
    serie_perceptores = serie_perceptores.reset_index(drop=True)
    serie_consumo = serie_consumo.reset_index(drop=True)

    # Para creación de columna Fecha:

    xlsx_files = [f for f in os.listdir(extract_dir) if f.endswith('.xlsx')]

    # Ruta al fichero esperado
    ruta_xlsx = os.path.join(extract_dir, xlsx_files[0])  # si hay solo uno

    # Leer el contenido
    df = pd.read_excel(ruta_xlsx, skiprows=3, usecols=[23, 24, 25])

    serie_ventas = serie_ventas.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_rendimientos = serie_rendimientos.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_importaciones = serie_importaciones.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_exportaciones = serie_exportaciones.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_ventas_construcción = serie_ventas_construcción.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_perceptores = serie_perceptores.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')
    serie_consumo = serie_consumo.merge(df, left_on='PERIODO', right_on='Id_periodo', how='inner')

    # Selección de variable

    serie_ventas = serie_ventas[['Literal', 'VALOR']].rename(columns={'VALOR': 'Ventas', 'Literal':'Fecha'})
    serie_rendimientos = serie_rendimientos[['Literal', 'VALOR']].rename(columns={'VALOR': 'Rendimientos', 'Literal':'Fecha'})
    serie_importaciones = serie_importaciones[['Literal', 'VALOR']].rename(columns={'VALOR': 'Importaciones', 'Literal':'Fecha'})
    serie_exportaciones = serie_exportaciones[['Literal', 'VALOR']].rename(columns={'VALOR': 'Exportaciones', 'Literal':'Fecha'})
    serie_ventas_construcción = serie_ventas_construcción[['Literal', 'VALOR']].rename(columns={'VALOR': 'Ventas_construccion', 'Literal':'Fecha'})
    serie_perceptores = serie_perceptores[['Literal', 'VALOR']].rename(columns={'VALOR': 'Perceptores', 'Literal':'Fecha'})
    serie_consumo = serie_consumo[['Literal', 'VALOR']].rename(columns={'VALOR': 'Ventas_consumo', 'Literal':'Fecha'})
    
    # Estandarización

    # Convertir columna 'Literal' a datetime
    serie_ventas['Fecha'] = pd.to_datetime(serie_ventas['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_rendimientos['Fecha'] = pd.to_datetime(serie_rendimientos['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_importaciones['Fecha'] = pd.to_datetime(serie_importaciones['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_exportaciones['Fecha'] = pd.to_datetime(serie_exportaciones['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_ventas_construcción['Fecha'] = pd.to_datetime(serie_ventas_construcción['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_perceptores['Fecha'] = pd.to_datetime(serie_perceptores['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    serie_consumo['Fecha'] = pd.to_datetime(serie_consumo['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)
    
    # Convertir 'VALOR' a float (sustituyendo coma por punto)
    serie_ventas['Ventas'] = serie_ventas['Ventas'].str.replace(',', '.', regex=False).astype(float)
    serie_rendimientos['Rendimientos'] = serie_rendimientos['Rendimientos'].str.replace(',', '.', regex=False).astype(float)
    serie_importaciones['Importaciones'] = serie_importaciones['Importaciones'].str.replace(',', '.', regex=False).astype(float)
    serie_exportaciones['Exportaciones'] = serie_exportaciones['Exportaciones'].str.replace(',', '.', regex=False).astype(float)
    serie_ventas_construcción['Ventas_construccion'] = serie_ventas_construcción['Ventas_construccion'].str.replace(',', '.', regex=False).astype(float)
    serie_perceptores['Perceptores'] = serie_perceptores['Perceptores'].str.replace(',', '.', regex=False).astype(float)
    serie_consumo['Ventas_consumo'] = serie_consumo['Ventas_consumo'].str.replace(',', '.', regex=False).astype(float)

    
    return serie_ventas, serie_rendimientos, serie_importaciones, serie_exportaciones, serie_ventas_construcción, serie_consumo


In [60]:
# Consumo eléctrico

def obtener_ire_general(start_year, end_year, geo_limit="peninsular", geo_ids="8741", time_trunc="month", lang="en"):
    
    all_data = []

    for year in range(start_year, end_year + 1):
        url = f"https://apidatos.ree.es/{lang}/datos/demanda/ire-general"
        params = {
            "start_date": f"{year}-01-01T00:00",
            "end_date": f"{year}-12-31T23:59",
            "time_trunc": time_trunc,
            "geo_trunc": "electric_system",
            "geo_limit": geo_limit,
            "geo_ids": geo_ids
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error {response.status_code} al consultar el año {year}")
            continue

        try:
            data = response.json()
            valores = data["included"][0]["attributes"]["values"]
            df = pd.DataFrame(valores)
            df = df[["datetime", "value"]].rename(columns={"datetime": "Fecha", "value": "Consumo_electrico"})
            df["Fecha"] = df["Fecha"].str[:10]
            
            all_data.append(df[["Fecha", "Consumo_electrico"]])
        
        except Exception as e:
            print(f"Error procesando los datos del año {year}: {e}")
    
    if all_data:
        data_final = pd.concat(all_data, ignore_index=True)
    else:
        data_final = pd.DataFrame(columns=["Fecha", "Consumo_electrico"])

    data_final["Fecha"] = pd.to_datetime(data_final["Fecha"], format="%Y-%m-%d")
    data_final["Consumo_electrico"] = data_final["Consumo_electrico"].astype(float)
    
    return data_final


In [ ]:
def obtener_series_bde():
    url = "https://app.bde.es/bierest/resources/srdatosapp/listaSeries?idioma=en&series=D_1TI2THITA&rango=MAX"

    response = requests.get(url)

    data_json = response.json() 
    serie = data_json[0]
    fechas = serie["fechas"]
    valores = serie["valores"]

    # Crear DataFrame
    df = pd.DataFrame({
        "Fecha": [f[:10] for f in fechas], 
        "Credito": valores
    })

    # Ordenar por fecha ascendente
    df = df.sort_values("Fecha").reset_index(drop=True)

    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")
    df["Credito"] = df["Credito"].astype(float)
    
    return df

In [62]:
# Obtener todas las series
serie_afiliados = obtener_afiliados_seguridad_social()   # Trabajadores en alta laboral en la Seguridad Social. Media mensual*. CVEC

series_ine = obtener_ine_series() # Obtención de variables del INE

serie_IASS = pd.read_excel("IASS.xlsx")

serie_IPI = pd.read_excel("IPI.xlsx")

serie_PIB = pd.read_excel("PIB.xlsx")

serie_ocupados = pd.read_excel("EPA.xlsx")

serie_IPI_consumo = pd.read_excel("IPI_consumo.xlsx")

serie_IPI_equipo = pd.read_excel("IPI_equipo.xlsx")

serie_Retail = pd.read_excel("Retail.xlsx")

serie_Entradas_Turismo = pd.read_excel("Entradas_turistas.xlsx")

serie_Compraventas = pd.read_excel("Compraventa.xlsx")

serie_PIB = pd.read_excel("PIB.xlsx")

serie_cemento = obtener_series_cemento()

serie_ventas, serie_rendimientos, serie_importaciones, serie_exportaciones, serie_ventas_construcción, serie_consumo = obtener_series_AEAT() # Obtención variables AEAT

services_pmi, manufacturing_pmi = obtener_series_PMI() # Obtención variables Markit

series_consumo_electrico = obtener_ire_general(1990, 2025)

serie_credito = obtener_series_bde()


Guardado: IPI.xlsx
Guardado: EPA.xlsx
Guardado: PIB.xlsx
Guardado: IASS.xlsx
Guardado: IPI_consumo.xlsx
Guardado: IPI_equipo.xlsx
Guardado: Retail.xlsx
Guardado: Entradas_turistas.xlsx
Guardado: Construccion_indice.xlsx
Guardado: Compraventa.xlsx
Guardado: PIB.xlsx


c:\Users\diaro\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'industria.gob.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\diaro\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'industria.gob.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Descarga cancelada. Se mantiene el archivo existente.
Archivo extraído: BdDVesge_extract\Diseño de registro tablas aux .xlsx
Archivo extraído: BdDVesge_extract\Vesge_pub_indexada.txt
Archivos .txt encontrados: ['Vesge_pub_indexada.txt']
Error 500 al consultar el año 1990
Error 500 al consultar el año 1991
Error 500 al consultar el año 1992
Error 500 al consultar el año 1993
Error 500 al consultar el año 1994
Error 500 al consultar el año 1995
Error 500 al consultar el año 1996
Error 500 al consultar el año 1997
Error 500 al consultar el año 1998
Error 500 al consultar el año 1999
Error 500 al consultar el año 2000
Error 500 al consultar el año 2001
Error 500 al consultar el año 2002
Error 500 al consultar el año 2003
Error 500 al consultar el año 2004
Error 500 al consultar el año 2005
Error 500 al consultar el año 2006
Error 500 al consultar el año 2007
Error 500 al consultar el año 2008
Error 500 al consultar el año 2009


In [63]:
# Manual:

serie_World_imports = pd.read_excel("World Trade in Goods.xlsx", usecols=['Fecha', 'World_Imports'])

serie_World_exports = pd.read_excel("World Trade in Goods.xlsx", usecols=['Fecha', 'World_Exports'])


In [64]:
# Eurostat

serie_confidence_indicator = eurostat.get_data('ei_bsco_m')

my_filter_pars = {'freq': 'M', 's_adj': 'SA', 'geo': 'ES', 'indic' : 'BS-CSMCI'}

serie_confidence_indicator_pre = eurostat.get_data_df('ei_bsco_m', filter_pars = my_filter_pars)

serie_confidence_indicator = serie_confidence_indicator_pre.melt(
    id_vars=['freq', 'indic', 's_adj', 'unit', 'geo\\TIME_PERIOD'],
    var_name='Fecha',
    value_name='Consumer_Confidence'
)

serie_confidence_indicator['Fecha'] = pd.to_datetime(serie_confidence_indicator['Fecha'], format="%Y-%m", errors='coerce') + pd.offsets.MonthBegin(0)

serie_confidence_indicator = serie_confidence_indicator[['Fecha', 'Consumer_Confidence']].dropna()

In [65]:
# Unión de variables

dfs = [serie_PIB, serie_IASS, serie_cemento, serie_IPI, serie_ocupados, serie_ventas, serie_exportaciones, serie_importaciones, serie_rendimientos, serie_ventas_construcción, serie_consumo, services_pmi, manufacturing_pmi, serie_afiliados, serie_IPI_consumo, serie_IPI_equipo, serie_Retail, serie_Entradas_Turismo, serie_ventas_construcción, serie_Compraventas, serie_World_imports, serie_World_exports, serie_confidence_indicator, series_consumo_electrico, serie_credito]

for i in dfs:
    print(i)
    print(i.dtypes)



         Fecha       PIB
0   1995-01-01   67.0160
1   1995-04-01   67.4578
2   1995-07-01   67.7565
3   1995-10-01   68.2696
4   1996-01-01   68.6618
..         ...       ...
116 2024-01-01  118.6042
117 2024-04-01  119.5708
118 2024-07-01  120.4238
119 2024-10-01  121.2817
120 2025-01-01  121.9704

[121 rows x 2 columns]
Fecha    datetime64[ns]
PIB             float64
dtype: object
         Fecha     IASS
0   2000-01-01   60.522
1   2000-02-01   60.292
2   2000-03-01   57.833
3   2000-04-01   62.111
4   2000-05-01   61.770
..         ...      ...
297 2024-10-01  126.154
298 2024-11-01  125.752
299 2024-12-01  127.982
300 2025-01-01  129.604
301 2025-02-01  129.718

[302 rows x 2 columns]
Fecha    datetime64[ns]
IASS            float64
dtype: object
         Fecha       Cemento
0   1992-01-01  2.036105e+06
1   1992-02-01  2.154241e+06
2   1992-03-01  2.435626e+06
3   1992-04-01  2.184208e+06
4   1992-05-01  2.281220e+06
..         ...           ...
394 2024-11-01  1.348144e+06
395 2024

In [66]:
df_total = reduce(lambda left, right: pd.merge(left, right, on='Fecha', how='outer'), dfs)

# Ordenar por fecha
df_merged = df_total.sort_values('Fecha').reset_index(drop=True)

In [67]:
df_merged.columns

Index(['Fecha', 'PIB', 'IASS', 'Cemento', 'IPI', 'EPA', 'Ventas',
       'Exportaciones', 'Importaciones', 'Rendimientos',
       'Ventas_construccion_x', 'Ventas_consumo', 'PMI_servicios',
       'PMI_manufactura', 'Afiliados_total', 'IPI_consumo', 'IPI_equipo',
       'Retail', 'Entradas_turistas', 'Ventas_construccion_y', 'Compraventa',
       'World_Imports', 'World_Exports', 'Consumer_Confidence',
       'Consumo_electrico', 'Credito'],
      dtype='object')

In [68]:
df_merged.head()

,Fecha,PIB,IASS,Cemento,IPI,EPA,Ventas,Exportaciones,Importaciones,Rendimientos,...,IPI_equipo,Retail,Entradas_turistas,Ventas_construccion_y,Compraventa,World_Imports,World_Exports,Consumer_Confidence,Consumo_electrico,Credito
0,1986-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.5,NaN,NaN
1,1986-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.5,NaN,NaN
2,1986-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.3,NaN,NaN
3,1986-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.1,NaN,NaN
4,1986-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.2,NaN,NaN


In [69]:
# Indicadores reducidos del modelo: 

df_indicadores_reducidos = df_merged[['Fecha', 'PIB','EPA', 'PMI_servicios', 'IPI', 'Ventas', 'Consumo_electrico', 'Credito', 'Afiliados_total', 'Importaciones', 'Consumer_Confidence']]

df_indicadores_reducidos = df_indicadores_reducidos[~df_indicadores_reducidos.drop(columns=["Fecha"]).isna().all(axis=1)]

In [70]:
df_indicadores_reducidos.head()

,Fecha,PIB,EPA,PMI_servicios,IPI,Ventas,Consumo_electrico,Credito,Afiliados_total,Importaciones,Consumer_Confidence
0,1986-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.5
1,1986-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.5
2,1986-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.3
3,1986-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.1
4,1986-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.2


In [71]:
df_indicadores_reducidos.tail()

,Fecha,PIB,EPA,PMI_servicios,IPI,Ventas,Consumo_electrico,Credito,Afiliados_total,Importaciones,Consumer_Confidence
464,2025-02-01,NaN,NaN,57.3,101.158,134.6576,103.082,3.97,2.119615e+07,126.8302,NaN
465,2025-03-01,NaN,NaN,54.9,NaN,NaN,115.064,3.88,2.135765e+07,NaN,NaN
466,2025-04-01,NaN,NaN,56.2,NaN,NaN,NaN,NaN,2.158864e+07,NaN,NaN
467,2025-05-01,NaN,NaN,54.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,2025-06-01,NaN,NaN,53.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_indicadores_reducidos.to_excel("Indicadores_Muestra.xlsx", index=False)

In [73]:
df_indicadores_reducidos.to_csv("Indicadores_Muestra.csv", index=False)